THIS FILE PERFORMS THE AUTOMATIC SCORING OF DATA FOR THE 6 METRICS - TOXICITY, RATIONALITY, MUTUAL RESSPECT, EMOTION, MODERATOR PRESENCE AND DIVERSITY.

In [ ]:
import pandas as pd
import json
import os
import glob
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification
from detoxify import Detoxify
from flair.models import TextClassifier
from flair.data import Sentence
import re
import nltk
from nltk.corpus import stopwords


# you want to use detoxify for toxicity and there are a couple of other libraries that also can be used for mutual respect, rationality, sentiment, diversity
# each of these should be added as a score to the dataframe

LOAD THE DATA FOR ANALYSIS

In [ ]:
df = pd.read_csv('./../data/unlabelled-posts/ScienceUncensored_unlabelled_data.csv')
df.head(10) 

TOXICITY (BATCHES FOR LARGE DATASETS)

In [ ]:
import pandas as pd
from detoxify import Detoxify

# Assuming df is your DataFrame and already created
# Example: df = pd.DataFrame({'Text': ["sample text"] * 10000})

text_to_toxicity = df['Text'].astype(str).tolist()

def process_batch(start_idx, end_idx):
    batch_text = text_to_toxicity[start_idx:end_idx]
    batch_scores = Detoxify('original').predict(batch_text)['toxicity']
    return batch_scores

batch_size = 500
detoxify_scores = []
for start_idx in range(0, len(text_to_toxicity), batch_size):
    end_idx = min(start_idx + batch_size, len(text_to_toxicity))
    batch_scores = process_batch(start_idx, end_idx)
    detoxify_scores.extend(batch_scores)

# Apply square root transformation
df['Toxicity'] = detoxify_scores
df['Toxicity'] = df['Toxicity'].apply(lambda x: x**0.5)

# Display the results
df.head(10)


In [ ]:
df.to_csv('./../data/unlabelled-posts/ScienceUncensored_unlabelled_data.csv', index=False)

TOXICITY

In [ ]:
text_to_toxicity = df['Text'].astype(str).tolist()
detoxify_scores = Detoxify('original').predict(text_to_toxicity)['toxicity']
# use square root transformation for toxicity score applying it onthe detoxify_scores
df['Toxicity'] = detoxify_scores
df['Toxicity'] = df['Toxicity'].apply(lambda x: x**0.5)

df.head(10)

RATIONALITY

In [ ]:
def truncate_text(text, max_tokens=512):
    tokens = tokenizer(text, truncation=True, max_length=max_tokens, return_tensors="pt")
    truncated_text = tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)
    return truncated_text

tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model", max_length=512, padding=True, truncation=True)
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer) # cuda = 0,1 based on gpu availability

df['Truncated Text'] = df['Text'].apply(lambda x: truncate_text(x, max_tokens=512))
df['Rationality'] = classifier(df['Truncated Text'].tolist())
df = df.drop(columns=['Truncated Text'])

In [ ]:
def modify_rationality(row):
    rationality = row['Rationality']
    
    if isinstance(rationality, dict):
        label = rationality['label']
        score = rationality['score']

        if label == 'Non-biased':
            return score
        else:
            return 1 - score
    else:
        return row['Rationality']
    
df['Rationality'] = df.apply(modify_rationality, axis=1)
df.head(10)

MUTUAL RESPECT

In [ ]:
def truncate_text(text, max_tokens=512):
    tokens = tokenizer(text, truncation=True, max_length=max_tokens, return_tensors="pt")
    truncated_text = tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)
    return truncated_text

tokenizer = AutoTokenizer.from_pretrained("NOVA-vision-language/polite_bert")
respect_model = AutoModelForSequenceClassification.from_pretrained("NOVA-vision-language/polite_bert")
classifier = pipeline('text-classification', model=respect_model, tokenizer=tokenizer)

df['Truncated Text'] = df['Text'].apply(lambda x: truncate_text(x, max_tokens=tokenizer.model_max_length))
df['Mutual Respect'] = classifier(df['Truncated Text'].tolist())
df = df.drop(columns=['Truncated Text'])

In [ ]:
def modify_mutual_respect(row):
    mutual_respect = row['Mutual Respect']
    
    if isinstance(mutual_respect, dict):
        label = mutual_respect['label']
        score = mutual_respect['score']

        if label == 'POLITE':
            return score
        elif label == 'SOMEWHAT_POLITE':
            return score * 0.7
        elif label == 'NEUTRAL':
            return score * 0.5
        else:
            return 1 - score
    else:
        return row['Mutual Respect']

df['Mutual Respect'] = df.apply(modify_mutual_respect, axis=1)
df.head(10)

EMOTION

In [ ]:
classifier = TextClassifier.load('en-sentiment')

def predict_sentiment(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    return sentence.labels[0].score

df['Emotion'] = df['Text'].apply(predict_sentiment)

In [ ]:
# use flair to predict the emotion score of the text
classifier = TextClassifier.load('en-sentiment')
def predict_sentiment(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    # if the value is NEGATIVE then we take - value if the value is POSITIVE then we just keep that value
    if sentence.labels[0].value == 'NEGATIVE':
        return sentence.labels[0].score* -1
    else:
        return sentence.labels[0].score

emotion = df['Text'].apply(predict_sentiment)
scaled_emotion = (emotion+1)/2 # scale the emotion scores to be between 0 and 1
df['Emotion'] = scaled_emotion

df.head(10)

MODERATOR PRESENCE

In [ ]:
with open('./../data/utils/moderators.json', 'r') as f:  # ensure the path to json of moderators for all subreddits is correct
    moderators_dict = json.load(f)

def is_moderator(author, subreddit='formula1'): # ensure subreddit is the correct subreddit
    return 1 if author in moderators_dict.get(subreddit, []) or author == 'AutoModerator' else 0

df['Moderator'] = df['Author'].apply(lambda x: is_moderator(x))
df.head(10)

DIVERSITY

In [ ]:
author_post_pairs = set()

def is_first_comment(post_id, author):
    pair = (post_id, author)
    if pair in author_post_pairs:
        return 0
    else:
        author_post_pairs.add(pair)
        return 1

df['Diversity'] = df.apply(lambda row: is_first_comment(row['PostID'], row['Author']), axis=1)
df.head(10)


SAVE DATAFRAME TO CSV

In [ ]:
# adjust path to the correct subreddit name & location
df.to_csv('./../data/unlabelled-posts/fauxmoi_unlabelled_data.csv', index=False)

SINGLE FUNCTION TO SCORE POSTS

In [ ]:
df = pd.read_csv('./../data/validation-posts/science_data.csv')
df.head(10) 

In [ ]:
# Toxicity Scoring

text_to_toxicity = df['Text'].astype(str).tolist()
detoxify_scores = Detoxify('original').predict(text_to_toxicity)['toxicity']
df['Toxicity'] = detoxify_scores
df['Toxicity'] = df['Toxicity'].apply(lambda x: x**0.5)

# Rationality Scoring

def truncate_text(text, max_tokens=512):
    tokens = tokenizer(text, truncation=True, max_length=max_tokens, return_tensors="pt")
    truncated_text = tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)
    return truncated_text

tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model", max_length=512, padding=True, truncation=True)
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer) # cuda = 0,1 based on gpu availability

df['Truncated Text'] = df['Text'].apply(lambda x: truncate_text(x, max_tokens=512))
df['Rationality'] = classifier(df['Truncated Text'].tolist())

def modify_rationality(row):
    rationality = row['Rationality']
    
    if isinstance(rationality, dict):
        label = rationality['label']
        score = rationality['score']

        if label == 'Non-biased':
            return score
        else:
            return 1 - score
    else:
        return row['Rationality']
    
df['Rationality'] = df.apply(modify_rationality, axis=1)

# Mutual Respect Scoring

tokenizer = AutoTokenizer.from_pretrained("NOVA-vision-language/polite_bert")
respect_model = AutoModelForSequenceClassification.from_pretrained("NOVA-vision-language/polite_bert")
classifier = pipeline('text-classification', model=respect_model, tokenizer=tokenizer)

df['Mutual Respect'] = classifier(df['Truncated Text'].tolist())
df = df.drop(columns=['Truncated Text'])

def modify_mutual_respect(row):
    mutual_respect = row['Mutual Respect']
    
    if isinstance(mutual_respect, dict):
        label = mutual_respect['label']
        score = mutual_respect['score']

        if label == 'POLITE':
            return score
        elif label == 'SOMEWHAT_POLITE':
            return score * 0.7
        elif label == 'NEUTRAL':
            return score * 0.5
        else:
            return 1 - score
    else:
        return row['Mutual Respect']

df['Mutual Respect'] = df.apply(modify_mutual_respect, axis=1)

# Emotion Scoring

classifier = TextClassifier.load('en-sentiment')

def predict_sentiment(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    return sentence.labels[0].score

df['Emotion'] = df['Text'].apply(predict_sentiment)

classifier = TextClassifier.load('en-sentiment')
def predict_sentiment(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    if sentence.labels[0].value == 'NEGATIVE':
        return sentence.labels[0].score* -1
    else:
        return sentence.labels[0].score

emotion = df['Text'].apply(predict_sentiment)
scaled_emotion = (emotion+1)/2 
df['Emotion'] = scaled_emotion

# Moderator Scoring

with open('./../data/utils/moderators.json', 'r') as f:  # ensure the path to json of moderators for all subreddits is correct
    moderators_dict = json.load(f)

"""def is_moderator(author, subreddit): # ensure subreddit is the correct subreddit
    return 1 if author in moderators_dict.get(subreddit, []) else 0

df['Moderator'] = df.apply(lambda row: is_moderator(row['Author'], row['Subreddit']), axis=1)"""

def is_moderator(author, subreddit='science'): # ensure subreddit is the correct subreddit
    return 1 if author in moderators_dict.get(subreddit, []) else 0

df['Moderator'] = df['Author'].apply(lambda x: is_moderator(x))
df.head(10)

# Diversity Scoring

author_post_pairs = set()

def is_first_comment(post_id, author):
    pair = (post_id, author)
    if pair in author_post_pairs:
        return 0
    else:
        author_post_pairs.add(pair)
        return 1

df['Diversity'] = df.apply(lambda row: is_first_comment(row['PostID'], row['Author']), axis=1)

df.head(10)

In [ ]:
# adjust path to the correct subreddit name & location
df.to_csv('./../data/validation-posts/science_data.csv', index=False)